This notebook generates a table of intensities and colours for the visualiation of raw timsTOF data, and generates a JSON lookup file.

In [88]:
from matplotlib import colors, cm, pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import sqlite3

In [45]:
MINIMUM_PIXEL_INTENSITY = 1
MAXIMUM_PIXEL_INTENSITY = 1000

PIXELS_X = 1000
PIXELS_Y = 30

In [46]:
# create the colour map to convert intensity to colour
colour_map = plt.get_cmap('rainbow')
norm = colors.LogNorm(vmin=MINIMUM_PIXEL_INTENSITY, vmax=MAXIMUM_PIXEL_INTENSITY, clip=True)  # aiming to get good colour variation in the lower range, and clipping everything else

In [47]:
colours_l = []
for i in range(MINIMUM_PIXEL_INTENSITY,MAXIMUM_PIXEL_INTENSITY+1):
    ic = (i,) + tuple([c for c in colour_map(norm(i), bytes=False)])
    colours_l.append(ic)

In [48]:
colours_df = pd.DataFrame(colours_l, columns=['intensity','r','g','b','alpha'])

In [69]:
colours_df['r_byte'] = np.uint8(colours_df.r * 255.0)
colours_df['g_byte'] = np.uint8(colours_df.g * 255.0)
colours_df['b_byte'] = np.uint8(colours_df.b * 255.0)

In [70]:
colours_df.sample(n=5)

,intensity,r,g,b,alpha,r_byte,g_byte,b_byte
918,919,1.000000,0.036951,0.018479,1.0,255,9,4
942,943,1.000000,0.024637,0.012320,1.0,255,6,3
477,478,1.000000,0.326539,0.165554,1.0,255,83,42
151,152,0.958824,0.751332,0.412356,1.0,244,191,105
314,315,1.000000,0.494656,0.255843,1.0,255,126,65


In [71]:
im_array = np.zeros([PIXELS_Y+1, PIXELS_X+1, 3], dtype=np.uint8)

In [82]:
for r in zip(colours_df.intensity, colours_df.r_byte, colours_df.g_byte, colours_df.b_byte):
    x = r[0]
    im_array[:,x,:] = [r[1],r[2],r[3]]


In [85]:
im = Image.fromarray(im_array, 'RGB')


In [86]:
im.show()

In [89]:
# write out the colour table to a SQLite file
colourmap_file_name = '/Users/darylwilding-mcbride/Downloads/colourmap.sqlite'
db_conn = sqlite3.connect(colourmap_file_name)
colours_df[['intensity','r','g','b']].to_sql(name='colours', con=db_conn, if_exists='replace', index=False)
db_conn.close()